Date: Oct 25

Edits: 


1. change Y depression/no depression ratio: depression / no depression = 1/3

## goals: 

1. class imbalance:
	* Y  (copy the entire row including X, add noise to X) 
    * 拼3年的数据，让Y=1的数据～150， Y=0～150，选出共同的X 
    * 拼成一个大表：有x有y
2. missing value: Y - drop NaN, X - use mode for now
3. model: classification - logistic regression
4. features X: select all related variables from demo & questionnaire, general (age, income, gender, … )
 

## Import modules and libraries

In [311]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing 
from fancyimpute import KNN   
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from functools import reduce
%matplotlib inline

## Load data for Y

In [312]:
# Y: CIDDSCOR - depression score, target: Both males and females 20 YEARS - 39 YEARS
# Can't impute Y, missing Y must be dropped
# SEQN are built upon previous years, can vertically stack 3 depression datasets

df_depression1 = pd.read_sas('DEPRESSION/CIQMDEP.XPT')[['SEQN','CIDDSCOR']].dropna()  # 1:positive-50, 5:negative-664, missing-64
df_depression2 = pd.read_sas('DEPRESSION/CIQDEP_B.XPT')[['SEQN','CIDDSCOR']].dropna() # 1:positive-48, 5:negative-764, missing-116
df_depression3 = pd.read_sas('DEPRESSION/CIQDEP_C.XPT')[['SEQN','CIDDSCOR']].dropna() # 1:positive-50, 5:negative-641, missing-159
dfs_depression = pd.concat([df_depression1, df_depression2, df_depression3])

print(dfs_depression['CIDDSCOR'].value_counts())
dfs_depression = dfs_depression.sort_values(by=['CIDDSCOR'])

# change ratio positive/negative = 1/3 --> 1/5 = 148/(148*3) --> keep the first 592=(148*4) rows in the dataframe
dfs_depression = dfs_depression[:592]
dfs_depression

5.0    2069
1.0     148
Name: CIDDSCOR, dtype: int64


,SEQN,CIDDSCOR
317,13830.0,1.0
17,21225.0,1.0
651,17790.0,1.0
654,17831.0,1.0
318,13897.0,1.0
658,17902.0,1.0
27,10271.0,1.0
318,24821.0,1.0
199,2729.0,1.0
513,6581.0,1.0


## Load data for X
### Demo data

In [313]:
# CODE	QUESTION	VALUES
# RIAGENDR	gender	1,2
# RIDAGEYR	Age at Screening Adjudicated	0-84, 85
# RIDRETH1	Race/Ethnicity	1,2,3,4,5
# DMDCITZN	citizenship 	1,2	7,9
# DMDMARTL	Marital Status	1-6	77,99
# INDHHINC	Annual Household Income	1-13	77,99
# INDFMINC	Annual Family Income	1-13	77,99
# DMDEDUC	Education - Recode (old version)	1,2,3	7,9
# DMDHHSIZ	Total number of people in the Household	1-6,7

df_demo1 = pd.read_sas('DEMO/DEMO.XPT')[['SEQN','RIAGENDR','RIDAGEYR','RIDRETH1','DMDCITZN','DMDMARTL','INDHHINC','INDFMINC','DMDEDUC','DMDHHSIZ']]   
df_demo2 = pd.read_sas('DEMO/DEMO_B.XPT')[['SEQN','RIAGENDR','RIDAGEYR','RIDRETH1','DMDCITZN','DMDMARTL','INDHHINC','INDFMINC','DMDEDUC','DMDHHSIZ']] 
df_demo3 = pd.read_sas('DEMO/DEMO_C.XPT')[['SEQN','RIAGENDR','RIDAGEYR','RIDRETH1','DMDCITZN','DMDMARTL','INDHHINC','INDFMINC','DMDEDUC','DMDHHSIZ']] 
dfs_demo = pd.concat([df_demo1, df_demo2, df_demo3])

### Questionnaire data

In [314]:
df_alcohol1 = pd.read_sas('QUESTIONNAIRE/ALCOHOL/ALQ.XPT')[['SEQN','ALQ120Q','ALQ120U','ALQ130','ALQ140Q','ALQ140U','ALQ150']]   
df_alcohol2 = pd.read_sas('QUESTIONNAIRE/ALCOHOL/ALQ_B.XPT')[['SEQN','ALQ120Q','ALQ120U','ALQ130','ALQ140Q','ALQ140U','ALQ150']] 
df_alcohol3 = pd.read_sas('QUESTIONNAIRE/ALCOHOL/ALQ_C.XPT')[['SEQN','ALQ120Q','ALQ120U','ALQ130','ALQ140Q','ALQ140U','ALQ150']] 
dfs_alcohol = pd.concat([df_alcohol1,df_alcohol2,df_alcohol3])

df_pain_relievers1 = pd.read_sas('QUESTIONNAIRE/PAIN RELIEVERS/RXQ_ANA.XPT')[['SEQN','RXQ300','RXQ310','RXD320','RXQ330','RXD331Q','RXD331U']]   
df_pain_relievers1.columns = ['SEQN','RXD300','RXD310','RXD320','RXD330','RXD331Q','RXD331U']
df_pain_relievers2 = pd.read_sas('QUESTIONNAIRE/PAIN RELIEVERS/RXQANA_B.XPT')[['SEQN','RXD300','RXD310','RXD320','RXD330','RXD331Q','RXD331U']]   
df_pain_relievers3 = pd.read_sas('QUESTIONNAIRE/PAIN RELIEVERS/RXQANA_C.XPT')[['SEQN','RXD300','RXD310','RXD320','RXD330','RXD331Q','RXD331U']]   
dfs_pain_relievers = pd.concat([df_pain_relievers1,df_pain_relievers2,df_pain_relievers3])
# pain reliever data has multiple responses from same SEQN --> WHY?
# print(dfs_pain_relievers)
dfs_pain_relievers = dfs_pain_relievers.drop_duplicates(subset='SEQN',keep='first', inplace=False)
# print(dfs_pain_relievers)

df_drug_use1 = pd.read_sas('QUESTIONNAIRE/DRUG USE/DUQ.XPT')[['SEQN','DUQ100','DUQ110','DUQ120','DUQ130']]
df_drug_use2 = pd.read_sas('QUESTIONNAIRE/DRUG USE/DUQ_B.XPT')[['SEQN','DUQ100','DUQ110','DUQ120','DUQ130']]
df_drug_use3 = pd.read_sas('QUESTIONNAIRE/DRUG USE/DUQ_C.XPT')[['SEQN','DUQ100','DUQ110','DUQ120','DUQ130']]
dfs_drug_use = pd.concat([df_drug_use1,df_drug_use2,df_drug_use3])
# dfs_drug_use

# SMOKING - Adult Recent Tobacco Use & Youth Cigarette/Tobacco Use
df_smoking1 = pd.read_sas('QUESTIONNAIRE/SMOKING/SMQMEC.XPT')[['SEQN','SMQ640','SMQ650']]
df_smoking2 = pd.read_sas('QUESTIONNAIRE/SMOKING/SMQMEC_B.XPT')[['SEQN','SMQ640','SMQ650']]
df_smoking3 = pd.read_sas('QUESTIONNAIRE/SMOKING/SMQMEC_C.XPT')[['SEQN','SMQ640','SMQ650']]
# SMOKING - Cigarette/Tobacco - adult
df_smoking11 = pd.read_sas('QUESTIONNAIRE/SMOKING/SMQ.XPT')[['SEQN','SMD070','SMD075','SMD080','SMD090','SMD100NI']]
df_smoking22 = pd.read_sas('QUESTIONNAIRE/SMOKING/SMQ_B.XPT')[['SEQN','SMD070','SMD075','SMD080','SMD090','SMD100NI']]
df_smoking33 = pd.read_sas('QUESTIONNAIRE/SMOKING/SMQ_C.XPT')[['SEQN','SMD070','SMD075','SMD641','SMD650','SMD100NI']]
df_smoking33.columns = ['SEQN','SMD070','SMD075','SMD080','SMD090','SMD100NI']
# SMOKING - household smokers
df_smoking111 = pd.read_sas('QUESTIONNAIRE/SMOKING/SMQFAM.XPT')[['SEQN','SMD410','SMD415']]
df_smoking222 = pd.read_sas('QUESTIONNAIRE/SMOKING/SMQFAM_B.XPT')[['SEQN','SMD410','SMD415']]
df_smoking333 = pd.read_sas('QUESTIONNAIRE/SMOKING/SMQFAM_C.XPT')[['SEQN','SMD410','SMD415']]

df_social_support1 = pd.read_sas('QUESTIONNAIRE/SOCIAL SUPPORT/SSQ.XPT')[['SEQN','SSQ010','SSQ020A','SSQ050','SSQ040','SSQ060']]
df_social_support1.columns = ['SEQN','SSQ011','SSQ021A','SSQ051','SSQ041','SSQ061']
df_social_support2 = pd.read_sas('QUESTIONNAIRE/SOCIAL SUPPORT/SSQ_B.XPT')[['SEQN','SSD011','SSD021A','SSD051','SSD041','SSD061']]
df_social_support2.columns = ['SEQN','SSQ011','SSQ021A','SSQ051','SSQ041','SSQ061']
df_social_support3 = pd.read_sas('QUESTIONNAIRE/SOCIAL SUPPORT/SSQ_C.XPT')[['SEQN','SSQ011','SSQ021A','SSQ051','SSQ041','SSQ061']]
dfs_social_support = pd.concat([df_social_support1,df_social_support2,df_social_support3])
# dfs_social_support

df_occupation1 = pd.read_sas('QUESTIONNAIRE/OCCUPATION/OCQ.XPT')[['SEQN','OCQ180','OCD230','OCD240']]
df_occupation2 = pd.read_sas('QUESTIONNAIRE/OCCUPATION/OCQ_B.XPT')[['SEQN','OCD180','OCD230','OCD240']]
df_occupation2.columns = ['SEQN','OCQ180','OCD230','OCD240'] 
df_occupation3 = pd.read_sas('QUESTIONNAIRE/OCCUPATION/OCQ_C.XPT')[['SEQN','OCQ180','OCD230','OCD240']]
dfs_occuptation = pd.concat([df_occupation1,df_occupation2,df_occupation3])
# dfs_occuptation

df_panic_disorder1 = pd.read_sas('QUESTIONNAIRE/MENTAL HEALTH/CIQPANIC.XPT')[['SEQN','CIDPSCOR']]
df_panic_disorder2 = pd.read_sas('QUESTIONNAIRE/MENTAL HEALTH/CIQPAN_B.XPT')[['SEQN','CIDPSCOR']]
df_panic_disorder3 = pd.read_sas('QUESTIONNAIRE/MENTAL HEALTH/CIQPAN_C.XPT')[['SEQN','CIDPSCOR']]
dfs_panic_disorder = pd.concat([df_panic_disorder1,df_panic_disorder2,df_panic_disorder3])
# dfs_panic_disorder

df_general_anxiety_disorder1 = pd.read_sas('QUESTIONNAIRE/MENTAL HEALTH/CIQGAD.XPT')[['SEQN','CIDGSCOR']]
df_general_anxiety_disorder2 = pd.read_sas('QUESTIONNAIRE/MENTAL HEALTH/CIQGAD_B.XPT')[['SEQN','CIDGSCOR']]
df_general_anxiety_disorder3 = pd.read_sas('QUESTIONNAIRE/MENTAL HEALTH/CIQGAD_C.XPT')[['SEQN','CIDGSCOR']]
dfs_general_anxiety_disorder = pd.concat([df_general_anxiety_disorder1,df_general_anxiety_disorder2,df_general_anxiety_disorder3])
# dfs_general_anxiety_disorder

# dfs_pain_relievers



In [315]:
dfs =   [dfs_depression, dfs_demo, dfs_alcohol, 
        dfs_drug_use,dfs_general_anxiety_disorder, 
        dfs_occuptation,dfs_pain_relievers, 
        dfs_panic_disorder, dfs_social_support]

df = reduce(lambda left,right: pd.merge(left,right,on='SEQN',how='left'), dfs)
df

,SEQN,CIDDSCOR,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDMARTL,INDHHINC,INDFMINC,DMDEDUC,...,RXD320,RXD330,RXD331Q,RXD331U,CIDPSCOR,SSQ011,SSQ021A,SSQ051,SSQ041,SSQ061
0,13830.0,1.0,2.0,38.0,2.0,1.0,1.0,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
1,21225.0,1.0,2.0,26.0,3.0,2.0,6.0,7.0,5.0,3.0,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
2,17790.0,1.0,1.0,39.0,3.0,1.0,3.0,3.0,3.0,1.0,...,1.0,1.0,1.0,1.0,5.0,NaN,NaN,NaN,NaN,NaN
3,17831.0,1.0,2.0,26.0,3.0,1.0,1.0,8.0,8.0,2.0,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
4,13897.0,1.0,1.0,36.0,1.0,2.0,4.0,7.0,3.0,1.0,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
5,17902.0,1.0,1.0,32.0,4.0,1.0,1.0,11.0,11.0,1.0,...,666.0,2.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
6,10271.0,1.0,1.0,34.0,3.0,1.0,1.0,11.0,11.0,3.0,...,666.0,2.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
7,24821.0,1.0,2.0,39.0,3.0,1.0,5.0,5.0,2.0,1.0,...,666.0,2.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
8,2729.0,1.0,2.0,35.0,3.0,1.0,5.0,99.0,99.0,2.0,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
9,6581.0,1.0,2.0,34.0,3.0,1.0,1.0,8.0,8.0,2.0,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN


## Zeros

In [316]:
def clean_epsilon(cell):
    if cell - 0.0 < 0.0001:
        return 0
    return cell

df = df.applymap(clean_epsilon)

## Dealing with missing values

In [317]:
def treat_nan_as_another_category(df):
    print('Missing values: treat as another category')
    df = df.fillna(50)
    return df

def impute_nan_as_mode(df):
    print('Missing values: impute using mode')
    # drop the columns if all the elements inside the columns are NaN
    count_row = df.shape[0]
    dropped_cols = []
    for col in df:
        if df[col].isnull().sum() == count_row:
            dropped_cols.append(col)
            df = df.drop(columns = [col])
    print('Columns',dropped_cols,'are dropped because all the cells in these columns are NaN')
        
    my_imputer = Imputer(missing_values='NaN', strategy='most_frequent')
    data_with_imputed_values = my_imputer.fit_transform(df)           # type: 2D array
    data_with_imputed_values = pd.DataFrame(data_with_imputed_values) # change 2D array to dataframe
    data_with_imputed_values.columns = df.columns # give the newly created dataframe column names 
    df = data_with_imputed_values
    return df

def drop_nan(df):
    print('Missing values: rows with NaN are dropped')
    df = df.dropna()
    return df


'''Now we have three variables, the gender, the income and the level of depression that has missing values. 
We then assume that people of similar income and of same gender tend to have the same level of depression. 
For a given missing value, we will look at the gender of the person, its income, look for its k nearest 
neighbors and get their level of depression. We can then approximate the depression level of the person we wanted.
knn explained: https://www.dataquest.io/blog/k-nearest-neighbors-in-python/ '''


X1 = [[0], [1], [2], [3]] # X must be 2D array
y1 = [0, 0, 1, 1]

def impute_nan_using_KNN(X,y):    

    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(X, y) 
    print(neigh.predict([[2.5]]))

## data standardization

## Model

In [318]:
def LinearSVC(X_train, X_test, y_train, y_test):
    print('Model: svm.LinearSVC.')
    clf = svm.LinearSVC(C=1000,dual=False)
    clf.fit(X_train, y_train)
    train_predictions = clf.predict(X_train)
    test_predictions = clf.predict(X_test)
    return train_predictions, test_predictions

def LogisticRegressionModel(X_train, X_test, y_train, y_test):
    print('Model: Logistic Regression')
    lr = LogisticRegression(C=1,dual=False)
    lr.fit(X_train, y_train)
    train_predictions = lr.predict(X_train)
    test_predictions = lr.predict(X_test)
    return train_predictions, test_predictions

## RUN

In [319]:
def compute_accuracy(predict, actual):
        if len(predict) != len(actual):
            return 0
        correctness = 0
        for i in range(len(predict)):
            if predict[i] == actual[i]: 
                correctness += 1
            i += 1
        accuracy = correctness / len(predict)
        return accuracy
    
def run_prediction(df):

# pick one of the methods here to deal with missing values 
##########################################################
    # impute NaN values as mode of each col
    df = impute_nan_as_mode(df)
    # treat NaN as a new category
    # df = treat_nan_as_another_category(df)
    # drop rows that contain NaN
    # df = drop_nan(df)
##########################################################

    y = df['CIDDSCOR'].values
    columns_to_delete = ['SEQN', 'CIDDSCOR']
    remaining_df = df.drop(columns=columns_to_delete)
    X = remaining_df.values

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
    #train_predictions, test_predictions = LinearSVC(X_train, X_test, y_train, y_test)
    train_predictions, test_predictions = LogisticRegressionModel(X_train, X_test, y_train, y_test)

    print('Current features are:', remaining_df.columns.values)
    print('The training set accuracy is', compute_accuracy(train_predictions, y_train)*100,'%')
    print('The testing set accuracy is', compute_accuracy(test_predictions, y_test)*100,'%')
    print('------------------------------------------------------')

run_prediction(df)

Missing values: impute using mode
Columns ['SSQ011', 'SSQ021A', 'SSQ051', 'SSQ041', 'SSQ061'] are dropped because all the cells in these columns are NaN
Model: Logistic Regression
Current features are: ['RIAGENDR' 'RIDAGEYR' 'RIDRETH1' 'DMDCITZN' 'DMDMARTL' 'INDHHINC'
 'INDFMINC' 'DMDEDUC' 'DMDHHSIZ' 'ALQ120Q' 'ALQ120U' 'ALQ130' 'ALQ140Q'
 'ALQ140U' 'ALQ150' 'DUQ100' 'DUQ110' 'DUQ120' 'DUQ130' 'CIDGSCOR'
 'OCQ180' 'OCD230' 'OCD240' 'RXD300' 'RXD310' 'RXD320' 'RXD330' 'RXD331Q'
 'RXD331U' 'CIDPSCOR']
The training set accuracy is 77.16701902748414 %
The testing set accuracy is 81.5126050420168 %
------------------------------------------------------


- LinearSVC: original dp +/- ratio

        Missing values: impute using mode
        Model: svm.LinearSVC.
        Current features are: ['RIAGENDR' 'RIDAGEYR' 'RIDRETH1']
        The training set accuracy is 93.1190073322053 %
        The testing set accuracy is 94.14414414414415 %
        
- LogisticRegression: original dp +/- ratio
    
        Missing values: impute using mode
        Model: Logistic Regression
        Current features are: ['RIAGENDR' 'RIDAGEYR' 'RIDRETH1']
        The training set accuracy is 93.79582628313592 %
        The testing set accuracy is 91.44144144144144 %
        
- LogisticRegression:  dp +/- ratio = 1/3

        Missing values: impute using mode
        Model: Logistic Regression
        Current features are: ['RIAGENDR' 'RIDAGEYR' 'RIDRETH1' 'DMDCITZN' 'DMDMARTL' 'INDHHINC'
         'INDFMINC' 'DMDEDUC' 'DMDHHSIZ']
        The training set accuracy is 75.05285412262155 %
        The testing set accuracy is 72.26890756302521 %
        

# Questions:

- smote - class imbalance
- some X are categorical, some are a continuous range of values, ??

In [320]:
# # initialize the dataframe
# prev_df = dfs_depression
# # merge dfs, turn min values into zeros, and get rid of don't know & no response values
# def merge_and_clean(new_feature, prev_df, cstr):
#     dataset_name = list(new_feature)[0]
#     dataset_col = list(new_feature.values())[0]
#     cur_df = pd.read_sas(dataset_name)[['SEQN', dataset_col]]
#     cur_df[dataset_col] = cur_df[dataset_col].apply(clean_epsilon)
#     cur_df[dataset_col] = cur_df[dataset_col].apply(lambda x: constraints(x,cstr))
#     df = pd.merge(prev_df, cur_df, on='SEQN')
#     prev_df = df 
#     return prev_df

# def constraints(cell, cstr):
#     # forbidden values
#     if cell in cstr: return np.nan
#     else: return cell

